In [5]:
from geopy.geocoders import Nominatim
import pandas as pd

In [10]:
df = pd.read_csv('details.txt', delimiter=';')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31850 entries, 0 to 31849
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   location       31845 non-null  object 
 1   total_price    31849 non-null  object 
 2   price_per_sqm  29473 non-null  object 
 3   area           31849 non-null  object 
 4   rooms          31849 non-null  object 
 5   finished       6912 non-null   object 
 6   floor          31404 non-null  object 
 7   outside        23252 non-null  object 
 8   rent           15569 non-null  object 
 9   elevator       27957 non-null  object 
 10  built          24779 non-null  float64
 11  b_type         23639 non-null  object 
 12  link           31850 non-null  object 
dtypes: float64(1), object(12)
memory usage: 3.2+ MB
None
